<a href="https://colab.research.google.com/github/kimdonggyu2008/audio_synthesis/blob/main/keras_%ED%99%9C%EC%9A%A9%ED%95%9C_%EC%9D%B8%EC%BD%94%EB%8D%94%2C_%EB%94%94%EC%BD%94%EB%8D%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Conv2D, ReLU, BatchNormalization, \
    Flatten, Dense, Reshape, Conv2DTranspose, Activation
from tensorflow.keras import backend as K
import numpy as np

In [ ]:
class Autoencoder: #인코더 선언
  def __init__(self,
               input_shape,
               conv_filters,
               conv_kernels,
               conv_strides,
               latent_space_dim):
    self.input_shape=input_shape #인코더의 필터 갯수를 차례대로 선언, [28, 28, 1]
    self.conv_filters=conv_filters #각 레이어의 컨볼루션 필터 크기, [2, 4, 8] = 2*2,4*4,8*8
    self.conv_kernels=conv_kernels # 각 레이어의 커널(필터) 크기[3, 5, 3] = 3*3, 5*5, 3*3
    self.conv_strides=conv_strides #컨볼루션에 적용된 필터의 이동거리 [1, 2, 2]
    self.latent_space_dim=latent_space_dim #특징 공간 차원 수, 2

    self.encoder=None
    self.decoder=None
    self.model=None
    #기본 인코더, 디코더, 모델 비활성화

    self._num_conv_layers=len(conv_filters)#레이어 갯수
    self._shape_before_bottleneck=None

    self._build()

  def summary(self):
    self.encoder.summary()
    self.decoder.summary()

  def _build(self):
    self._build_encoder()#인코더 생성
    self._build_decoder()
    #self.build_autoencoder()

#디코더
  def _build_decoder(self): #디코더 생성, 구성요소 넣음
    decoder_input=self._add_decoder_input() #디코더의 입력값
    dense_layer=self._add_dense_layer(decoder_input) # 디코더 내의 완전연결층, 생성을 위함
    reshape_layer=self._add_reshape_layer(dense_layer) #레이어 크기 재조정(?)
    conv_transpose_layers=self._add_conv_transpose_layers(reshape_layer)#전치행렬로 변경
    decoder_output=self._add_decoder_output(conv_transpose_layers)# 디코더 생성값 출력
    self.decoder=Model(decoder_input,decoder_output,name="decoder")#입력값, 출력값의 형태를 가지고 모델 생성

  def _add_decoder_input(self):#입력값, 특징 공간 크기 지정
    return Input(shape=self.latent_space_dim,name="decoder_input")#디코더 입력값으로 지정

  def _add_dense_layer(self,decoder_input):#생성층 선언
    num_neurons=np.prod(self._shape_before_bottleneck)#보틀넥의 연결갯수 지정
    dense_layer=Dense(num_neurons,name="decoder_dense")(decoder_input)#
    return dense_layer

  def _add_reshape_layer(self,dense_layer): #
    return Reshape(self._shape_before_bottleneck)(dense_layer)#텐서의 모양을 입력된 크기로 변경

  def _add_conv_transpose_layers(self,x):#들어온 모든 값에 대해서 전치행렬화를 시킴
    for layer_index in reversed(range(1,self._num_conv_layers)):
      x=self._add_conv_transpose_layer(layer_index,x)
    return x

  def _add_conv_transpose_layer(self,layer_index,x):#컨볼루션 레이어 역순으로 재지정
    layer_num=self._num_conv_layers-layer_index # 레이어 순서를 역순으로 재지정
    conv_transpose_layer=Conv2DTranspose(#컨볼루션 구조 지정, 역순으로 가져와서 순서대로 넣음
        filters=self.conv_filters[layer_index],
        kernel_size=self.conv_kernels[layer_index],
        strides=self.conv_strides[layer_index],
        padding="same",
        name=f"decoder_conv_transpose_layer{layer_num}"
    )
    x=conv_transpose_layer(x)#현재 레이어 역순으로 된 걸로 새로저장
    x=ReLU(name=f"decoder_relu_{layer_num}")(x)#현재 레이어 활성화함수
    x=BatchNormalization(name=f"decoder_bn_{layer_num}")(x)#현재 레이어 정규화
    return x

  def _add_decoder_output(self,x):
    conv_transpose_layer=Conv2DTranspose(#뒤집힌 컨볼루션 레이어 선언
        filters=1,
        kernel_size=self.conv_kernels[0],
        strides=self.conv_strides[0],
        padding="same",
        name=f"decoder_conv_transpose_layer_{self._num_conv_layers}"
    )
    x=conv_transpose_layer(x)#현재 레이어 저장
    output_layer=Activation("sigmoid",name="sigmoid_layer")(x)#활성화 함수 적용함
    return output_layer#출력

#인코더
  def _build_encoder(self):#인코더 내부 채우기
    encoder_input=self._add_encoder_input()#인코더 입력칸 생성
    conv_layers=self._add_conv_layers(encoder_input)#입력값 컨볼루션에 넣음
    bottleneck=self._add_bottleneck(conv_layers)#보틀넥 적용
    self.encoder=Model(encoder_input,bottleneck,name="encoder")#인코더로 지정한 모델 생성

  def _add_encoder_input(self):#인코더 내부의 입력칸 생성
    return Input(shape=self.input_shape,name="encoder_input")#입력값 받아서 넣어줌

  def _add_conv_layers(self,encoder_input):#
    """인코더의 각 컨볼루션 레이어"""
    x=encoder_input#인코더 입력창
    for layer_index in range(self._num_conv_layers):
      x=self._add_conv_layer(layer_index,x)#레이어 갯수만큼 시행
    return x

  def _add_conv_layer(self,layer_index,x):#x번째, 컨볼루션 레이어 추가
    """conv2d+relu+batch normalization"""
    layer_number=layer_index+1#마지막 relu를 위한 레이어 추가로 넣음
    conv_layer=Conv2D( #컨볼루션 선언
        filters=self.conv_filters[layer_index],#필터 갯수
        kernel_size=self.conv_kernels[layer_index],#커널(필터) 크기
        strides=self.conv_strides[layer_index],#몇 칸씩 이동할건지
        padding="same",#동일 크기 유지
        name=f"encoder_conv_layer_{layer_number}"#이름 지정
    )
    x=conv_layer(x)#컨볼루션의 결괏값(?)
    x=ReLU(name=f"encoder_relu_{layer_number}")(x)#결괏값에 relu적용
    x=BatchNormalization(name=f"encoder_bn_{layer_number}")(x)# relu적용값에 정규화 적용
    return x #결과값 반화

  def _add_bottleneck(self,x):#Dense레이어 보틀넥 생성
    self._shape_before_bottleneck=K.int_shape(x)[1:]#backend 임포트한 것 적용, 평평하기 이전 데이터들 저장
    x=Flatten()(x) #차원 축소
    x=Dense(self.latent_space_dim,name="encoder_output")(x)
    return x


if __name__=="__main__":
  autoencoder=Autoencoder(
        input_shape=(28,28,1), #입력값
        conv_filters=(32, 64, 64, 64),
        conv_kernels=(3,3,3,3),
        conv_strides=(1,2,2,1),
        latent_space_dim=2
    )
  autoencoder.summary()


Model: "encoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder_input (InputLayer)  [(None, 28, 28, 1)]       0         
                                                                 
 encoder_conv_layer_1 (Conv  (None, 28, 28, 32)        320       
 2D)                                                             
                                                                 
 encoder_relu_1 (ReLU)       (None, 28, 28, 32)        0         
                                                                 
 encoder_bn_1 (BatchNormali  (None, 28, 28, 32)        128       
 zation)                                                         
                                                                 
 encoder_conv_layer_2 (Conv  (None, 14, 14, 64)        18496     
 2D)                                                             
                                                           